In [ ]:
import tensorflow as tf
import os
import sys
import json
import numpy as np

import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import schema_utils
import apache_beam as beam
import tempfile

In [ ]:
train_data = "/home/jugs/PycharmProjects/DelvifyFWD/others/iris/iris_training.csv"
test_data = "/home/jugs/PycharmProjects/DelvifyFWD/others/iris/iris_test.csv"

In [ ]:
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=train_data, target_dtype=np.int, features_dtype=np.float32)
testing_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=test_data, target_dtype=np.int, features_dtype=np.float32)

In [ ]:
training_set.target

In [ ]:
training_set.data

In [ ]:
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

In [ ]:
feature_columns

In [ ]:
'''
NUMERIC_FEATURE_KEYS = [
   'petal_lenght',
   'petal_width',
   'sepal_length',
   'sepal_width',
]
'''

In [ ]:
# RAW_DATA_FEATURE_SPEC = dict([(name, tf.io.FixedLenFeature([], tf.float32)) for name in NUMERIC_FEATURE_KEYS])

In [ ]:
# RAW_DATA_METADATA = dataset_metadata.DatasetMetadata(
#     schema_utils.schema_from_feature_spec(RAW_DATA_FEATURE_SPEC))

In [ ]:
# RAW_DATA_FEATURE_SPEC

In [ ]:
# RAW_DATA_METADATA

In [ ]:
'''
class MapAndFilterErrors(beam.PTransform):
  """Like beam.Map but filters out erros in the map_fn."""
  class _MapAndFilterErrorsDoFn(beam.DoFn):
    """Count the bad examples using a beam metric."""
    def __init__(self, fn):
        self._fn = fn
      # Create a counter to measure number of bad elements.
        self._bad_elements_counter = beam.metrics.Metrics.counter(
            'census_example', 'bad_elements')
    def process(self, element):
        try:
            yield self._fn(element)
        except Exception:  # pylint: disable=broad-except
        # Catch any exception the above call.
            self._bad_elements_counter.inc(1)

    def __init__(self, fn):
        self._fn = fn

    def expand(self, pcoll):
        return pcoll | beam.ParDo(self._MapAndFilterErrorsDoFn(self._fn))
'''

In [ ]:
'''
def transform_data(train_data_file, test_data_file, working_dir):
    def preprocessing_fn(inputs):
        outputs = inputs.copy()
        for key in NUMERIC_FEATURE_KEYS:
            outputs[key] = tft.scale_to_0_1(outputs[key])
        
        table_keys = ['a', 'b', 'c']
        initializer = tf.lookup.KeyValueTensorInitializer(
          keys=table_keys,
          values=tf.cast(tf.range(len(table_keys)), tf.int64),
          key_dtype=tf.string,
          value_dtype=tf.int64)
        table = tf.lookup.StaticHashTable(initializer, default_value=-1)
        outputs[LABEL_KEY] = table.lookup(outputs[LABEL_KEY])
        return outputs
    
    with beam.Pipeline() as pipeline:
        with tft_beam.Context(temp_dir=tempfile.mktemp()):
            ordered_columns = [
              'petal_lenght','petal_width','sepal_length','sepal_width',
          ]
        
            converter = tft.coders.CsvCoder(ordered_columns, RAW_DATA_METADATA.schema)

            raw_data = (
              pipeline
              | "ReadTrainData" >> beam.io.ReadFromText(train_data_file)
              | "FixCommasTrainData" >> beam.Map(lambda line: line.replace(', ', ',')))
            

            raw_dataset = (raw_data, RAW_DATA_METADATA)
            transformed_dataset , transform_fn = (raw_dataset | tft_beam.AnalyzeAndTransformDataset(preprocessing_fn))
            transformed_data, transformed_metadata = transformed_dataset

            transformed_data_coder = tft.coders.ExampleProtoCoder(transformed_metadata.schema)

            _ = (
              transformed_data
              | "EncodeTrainData" >> beam.Map(transformed_data_coder.encode)
              | "WriteTrainData" >> beam.io.WriteToTFRecord(os.path.join(working_dir, TRANSFORMED_TRAIN_DATA_FILEBASE))
               )

            raw_test_data = (
              pipeline
              | "ReadTestData" >> beam.io.ReadFromText(test_data_file, skip_header_lines=1)
              | "FixCommasTestData" >> beam.Map(lambda line: line.replace(", ", ","))
              | "RemoveTrailingPeriodsTestData" >> beam.Map(lambda line: line[:-1])
              | "DecodeTestData" >> MapAndFilterErrors(converter.decode)
             )

            raw_test_dataset = (raw_test_data, RAW_DATA_METADATA)
            trasformed_test_dataset = ((raw_test_dataset, transform_fn) | tft_beam.TransformDataset())
            transformed_test_data, _ = trasformed_test_dataset

            _ = (
              transformed_test_data
              | "EncodeTestData" >> beam.Map(transformed_data_coder.encode)
              | "WriteTestData" >> beam.io.WriteToTFRecord(os.path.join(working_dir, TRANSFORMED_TEST_DATA_FILEBASE))
            )

            _ = (
              transform_fn
              | "WriteTransformFn" >> tft_beam.WriteTransformFn(working_dir)
            )
    
'''

In [ ]:
'''
def _make_training_input_fn(tf_transform_output, transformed_examples, batch_size):
    def input_fn():
        dataset = tf.data.experimental.make_batched_features_dataset(
            file_pattern=transformed_examples,
            batch_size=batch_size,
            features=tf_transform_output.transformed_feature_spec(),
            reader=tf.data.TFRecordDataset,
            shuffle=True
        )
        transformed_features = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
        transformed_labels = transformed_features.pop(LABEL_KEY)
        return transformed_features, transformed_labels
    return input_fn
'''

In [ ]:
'''
def _make_training_input_fn(tf_transform_output, transformed_examples, batch_size):
    def input_fn():
        dataset = tf.data.experimental.make_batched_features_dataset(
            file_pattern=transformed_examples,
            batch_size=batch_size,
            features=tf_transform_output.transformed_feature_spec(),
            reader=tf.data.TFRecordDataset,
            shuffle=True
        )
        transformed_features = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
        transformed_labels = transformed_features.pop(LABEL_KEY)
        return transformed_features, transformed_labels
    return input_fn
'''

In [ ]:
'''
def _make_training_input_fn(tf_transform_output, transformed_examples, batch_size):
    def input_fn():
        dataset = tf.data.experimental.make_batched_features_dataset(
            file_pattern=transformed_examples,
            batch_size=batch_size,
            features=tf_transform_output.transformed_feature_spec(),
            reader=tf.data.TFRecordDataset,
            shuffle=True
        )
        transformed_features = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
        transformed_labels = transformed_features.pop(LABEL_KEY)
        return transformed_features, transformed_labels
    return input_fn
'''

In [ ]:
'''
def train_and_evaluate(working_dir, ):
    tf_transformed_output = tft.TFTransformOutput(working_dir)
    run_config = tf.estimator.RunConfig()

    estimator = tf.estimator.LinearClassifier(
        feature_columns=get_feature_columns(tf_transformed_output),
        config=run_config,
        loss_reduction=tf.losses.Reduction.SUM
    )

    train_input_fn = _make_training_input_fn(
        tf_transformed_output,
        os.path.join(working_dir, TRANSFORMED_TRAIN_DATA_FILEBASE + '*'),
        batch_size=TRAIN_BATCH_SIZE
    )
    estimator.train(input_fn=train_input_fn, max_steps=100)

    eval_input_fn = _make_training_input_fn(
        tf_transformed_output,
        os.path.join(working_dir, TRANSFORMED_TEST_DATA_FILEBASE + '*'),
        batch_size=1
    )

    serving_input_fn = _make_serving_input_fn(tf_transformed_output)
    exported_model_dir = os.path.join(working_dir, EXPORTED_MODEL_DIR)
    estimator.export_savedmodel(exported_model_dir, serving_input_fn)

    return estimator.evaluate(input_fn=eval_input_fn, steps=num_test_instances)

'''

In [ ]:
'''
working_dir = '/tmp/iris'
train_data = "/home/jugs/PycharmProjects/DelvifyFWD/others/iris/iris_training.csv"
test_data = "/home/jugs/PycharmProjects/DelvifyFWD/others/iris/iris_test.csv"

transform_data(train_data, test_data, working_dir)

'''

In [ ]:
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                        hidden_units=[10,30,10], 
                                        n_classes=3, model_dir="/tmp/iris/iris_model")

In [ ]:
classifier.train(input_fn=)

In [ ]:
classifier.fit(x=training_set.data, y=training_set.target, max_steps=20000)

In [ ]:
def _make_training_input_fn():
    def input_fn():
        dataset = tf.data.experimental.make_batched_features_dataset(
            
        )

In [ ]:
accuracy_score = classifier.evaluate(x=testing_set.data, y=testing_set.target)['accuracy']

In [ ]:
print('Accuracy: {0:f}'.format(accuracy_score))

In [ ]:
new_samples = np.array([[6.4, 3.2, 4.5, 1.5], [6.4, 2.8, 5.6, 2.7], [5.8, 3.1, 5.0, 1.7]], dtype=float)

y = list(classifier.predict(new_samples, as_iterable=True))
print('Predictions: {}'.format(str(y)))

In [ ]:
classifier.save()

In [ ]:
sess = tf.Session()
tf.saved_model.simple_save(sess)

# New Version

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jugs/PycharmProjects/myvenv/lib/python3.6/s

In [2]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [3]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

In [4]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [5]:
train_y = train.pop('Species')
test_y = test.pop('Species')

# The label column has now been removed from the features.
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [6]:
def input_evaluation_set():
    features = {'SepalLength': np.array([6.4, 5.0]),
                'SepalWidth':  np.array([2.8, 2.3]),
                'PetalLength': np.array([5.6, 3.3]),
                'PetalWidth':  np.array([2.2, 1.0])}
    labels = np.array([2, 1])
    return features, labels

In [7]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [8]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [9]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9cv7ngir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f99d7890a90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp9cv7ngir/model.ckpt.
INFO:tensorflow:loss = 427.8204, step = 1
INFO:tensorflow:global_step/sec: 597.94
INFO:tensorflow:loss = 41.41667, step = 101 (0.168 sec)
INFO:tensorflow:glo

In [11]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-10-31T18:30:54Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/tmp9cv7ngir/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-10-31-18:30:54
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.96666664, average_loss = 0.081359774, global_step = 5000, loss = 2.4407933
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmp9cv7ngir/model.ckpt-5000

Test set accuracy: 0.967



In [12]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

In [13]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9cv7ngir/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (99.8%), expected "Setosa"
Prediction is "Versicolor" (100.0%), expected "Versicolor"
Prediction is "Virginica" (100.0%), expected "Virginica"


In [ ]:
def serving_input_receiver_fn():
    receiver_tensors = {
        'sepal_length': tf.placeholder(tf.float32, [None, 1]),
        'sepal_width': tf.placeholder(tf.float32, [None, 1]),
        'petal_length': tf.placeholder(tf.float32, [None, 1]),
        'petal_width': tf.placeholder(tf.float32, [None, 1]),
    }

    # Convert give inputs to adjust to the model.
    features = {
        INPUT_FEATURE: tf.concat([
            receiver_tensors['sepal_length'],
            receiver_tensors['sepal_width'],
            receiver_tensors['petal_length'],
            receiver_tensors['petal_width']
        ], axis=1)
    }
    return tf.estimator.export.ServingInputReceiver(receiver_tensors=receiver_tensors,
                                                    features=features)

In [16]:
classifier.export_saved_model("/tmp/iris", serving_input_receiver_fn=serving_input_receiver_fn)

NameError: name 'serving_input_receiver_fn' is not defined